In [1]:
import json
import os
import time

from datetime import datetime

from os.path import dirname, realpath
import sys
sys.path.append(dirname(dirname(realpath(os.getcwd()))))
import pickle
import torch
import multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
from collections import Counter
from copy import deepcopy
import datetime
import tqdm
import numpy as np
import random
from collections import Counter
sys.path.append("./src/")


import datetime
def parse_date(date_str):
    if date_str == 'NA' or date_str is None:
        return datetime.datetime(9999,1,1,0,0)
    else:
        if len(date_str) == 10:
            format_str = '%Y-%m-%d'
        else:
            raise Exception("Format for {} not recognized!".format(date_str))
    return datetime.datetime.strptime(date_str, format_str)

In [43]:
from bisect import bisect_left
from itertools import compress


In [152]:
VTE_path = 'Y:\\Fillmore_Cancer\\cat\\Users\\data\\derived\\vte_ml\\vte_ml\\data_for_chunlei'
train_path = os.path.join(VTE_path, 'A2_CMP\\test2\\test.json')
#test_path = os.path.join(VTE_path, 'A2_CMP\\XPheLab_CMP.json')
train_path = os.path.join(VTE_path, 'composite\\test\\test.json')


In [153]:
X = json.load(open(os.path.join(train_path), 'r'))

In [154]:
import pandas as pd

In [155]:
import statistics

In [156]:
import numpy as np

In [157]:
import pdb
import argparse

END_OF_TIME_DATE = datetime.datetime(2020, 12, 31, 0, 0)
OUTCOME_CODE = ['VTE']
BASELINE = []

UNK_TOKEN = '<UNK>'
PAD_TOKEN = '<PAD>'
MAX_TIME_EMBED_PERIOD_IN_DAYS = 12 * 365
MIN_TIME_EMBED_PERIOD_IN_DAYS = 10

SUMMARY_MSG = "Contructed BTH Disease Progression DiseaseRisk {} dataset with {} records, {} patients, and the following class balance \n {}"
NO_DATASET_ERR = "Dataset {} not in DATASET_REGISTRY! Available datasets are {}"
DATASET_REGISTRY = {}
NUM_PICKLES = 50
UNK_TOKEN = '<UNK>'
PAD_TOKEN = '<PAD>'
NO_OP_TOKEN = '_'

parser = argparse.ArgumentParser(description='DiskRisk Classifier')


In [158]:
  def find_closest_date_before(reference_date, dates):
        index = bisect_left(dates, reference_date)
        if index == 0:
            return None  # No date before the reference date
        return index - 1


In [159]:
#args = parse_args()
#args = parser.parse_args()



In [160]:
# these are VTE patients
patient_id = '1004562085'
#patient_id = '1003839942'
#patient_id = '1003839942'

In [161]:
def process_events1(events, end_date=None):
    for event in events:
        if 'admdate' in event.keys():
            event['admit_date'] = parse_date(event['admdate'])
        else:
            events.remove(event)
    if end_date is not None:
        start_date, start_admid = min([(e['admit_date'], e['admid']) for e in events])
        year_range = (end_date - start_date).days // 365
        for year in range(year_range):
            events.append(get_no_op_event1(start_date, year, start_admid))
    events = sorted(events, key=lambda event: event['admit_date'])
    return events


def process_events2(events, end_date=None):
    for event in events:
        if event['admdate'] is not None:
            # test = event['admdate']
            event['admit_date'] = parse_date(event['admdate'])
        else:
            events.remove(event)
    if end_date is not None:
        start_date, start_admid = min([(e['admit_date'], e['admid']) for e in events])
        year_range = (end_date - start_date).days // 365
        for year in range(year_range):
            events.append(get_no_op_event1(start_date, year, start_admid))
    events = sorted(events, key=lambda event: event['admit_date'])
    return events

def get_no_op_event1( start_date, year, admid_base):
    code = NO_OP_TOKEN
    new_date = start_date + datetime.timedelta(365 * year)
    event = {'admit_date': new_date, 'codes': code, 'hospital': 'n/a', 'admid': '{}_{}'.format(admid_base, year)}
    return event
def get_outcome_date1(events, index_date, end_of_date=END_OF_TIME_DATE):
    '''
    Looks through events to find date of outcome
    If multiple outcomes occur, use the first occurance date.
    args:
    - events: List of event dicts. Each dict must have a CODE and admit_date.
    '''
    outcome_events = [e for e in events if any(icd == e['codes'] for icd in OUTCOME_CODE)]

    if len(outcome_events) > 0:
        ever_develops_outcome = True
        time = min([e['admit_date'] for e in outcome_events])
    else:
        ever_develops_outcome = False
        time = end_of_date
    return ever_develops_outcome, time


In [162]:
patient_metadata = X

## USE R Dataframe to CHECK

In [180]:
patient_id = '1006513916'

In [181]:
#patient_id = '1003381732' # in both tests, good for case

In [182]:
#patient_id = '1001720232'  # for a control

In [183]:
obs_time_end = parse_date(patient_metadata[patient_id]['end_of_data'])
dob = parse_date(patient_metadata[patient_id]['birthdate'])            
index_date = parse_date(patient_metadata[patient_id]['indexdate'])
dx_date = parse_date(patient_metadata[patient_id]['dxdate'])
ks = patient_metadata[patient_id]['ks_mod_score']
sex = patient_metadata[patient_id]['gender']
bmi = patient_metadata[patient_id]['BMI']
race = patient_metadata[patient_id]['Race']

In [184]:
patient_dict = {'patient_id': patient_id}


In [185]:
 def get_outcome_date1(events, index_date, end_of_date=END_OF_TIME_DATE):
        '''
        Looks through events to find date of outcome
        If multiple outcomes occur, use the first occurance date.
        args:
        - events: List of event dicts. Each dict must have a CODE and admit_date.
        '''
        outcome_events = [e for e in events if any(icd == e['codes'] for icd in OUTCOME_CODE)]

        if len(outcome_events) > 0:
            ever_develops_outcome = True
            time = min([e['admit_date'] for e in outcome_events])
        else:
            ever_develops_outcome = False
            time = end_of_date
        return ever_develops_outcome, time

In [186]:
obs_time_end

datetime.datetime(2021, 3, 30, 0, 0)

In [187]:
feat_subgroup = {'birth_date': patient_metadata[patient_id]['birthdate']}
split_group = 'test'

In [188]:
events = process_events1(patient_metadata[patient_id]['events'], obs_time_end)
outcome, outcome_date = get_outcome_date1 ( events, index_date,   end_of_date=obs_time_end)

In [189]:
events

[{'admdate': '2000-11-30',
  'admid': '0008231784',
  'codes': '306',
  'admit_date': datetime.datetime(2000, 11, 30, 0, 0)},
 {'admit_date': datetime.datetime(2000, 11, 30, 0, 0),
  'codes': '_',
  'hospital': 'n/a',
  'admid': '0008231784_0'},
 {'admid': '0018838234',
  'codes': 'W2H2P2',
  'admdate': '2001-03-21',
  'admit_date': datetime.datetime(2001, 3, 21, 0, 0)},
 {'admit_date': datetime.datetime(2001, 11, 30, 0, 0),
  'codes': '_',
  'hospital': 'n/a',
  'admid': '0008231784_1'},
 {'admdate': '2002-10-15',
  'admid': '0009158936',
  'codes': '90.2',
  'admit_date': datetime.datetime(2002, 10, 15, 0, 0)},
 {'admdate': '2002-10-15',
  'admid': '0009158937',
  'codes': '90',
  'admit_date': datetime.datetime(2002, 10, 15, 0, 0)},
 {'admdate': '2002-10-22',
  'admid': '0009158934',
  'codes': '599.3',
  'admit_date': datetime.datetime(2002, 10, 22, 0, 0)},
 {'admdate': '2002-10-22',
  'admid': '0009158935',
  'codes': '599',
  'admit_date': datetime.datetime(2002, 10, 22, 0, 0)},


In [172]:
outcome

True

In [173]:
outcome_date

datetime.datetime(2009, 4, 17, 0, 0)

In [174]:
ev_dates = [events[i]['admit_date'] for i in range(len(events))]

In [175]:
t1 = index_date 
t2 = index_date + datetime.timedelta( 60 )  
t3 = index_date + datetime.timedelta( 120 )  
t4 = index_date + datetime.timedelta( 180 )  

In [176]:
t2

datetime.datetime(2009, 6, 12, 0, 0)

In [177]:
t3

datetime.datetime(2009, 8, 11, 0, 0)

In [132]:
t4

datetime.datetime(2009, 10, 10, 0, 0)

In [133]:
  
            selected_idx =  [find_closest_date_before(t1, ev_dates), 
                             find_closest_date_before(t2, ev_dates),
                             find_closest_date_before(t3, ev_dates),
                             find_closest_date_before(t4, ev_dates),]    

In [134]:
outcome_date

datetime.datetime(2009, 4, 17, 0, 0)

In [135]:
[t1, t2,t3,t4]

[datetime.datetime(2009, 4, 13, 0, 0),
 datetime.datetime(2009, 6, 12, 0, 0),
 datetime.datetime(2009, 8, 11, 0, 0),
 datetime.datetime(2009, 10, 10, 0, 0)]

In [139]:
sel_i =  [find_closest_date_before(t1, ev_dates),
          find_closest_date_before(t2, ev_dates),
          find_closest_date_before(t3, ev_dates),
          find_closest_date_before(t4, ev_dates),]    
valid =  [i<outcome_date for i in [t1, t2,t3,t4]]
selected_idx = list( compress(sel_i, valid))

In [140]:
sel_i

[761, 903, 904, 904]

In [141]:
selected_idx

[761]

In [142]:
 def get_label1 (event, index_date, month_endpoints):
        #event = patient['events'][idx]
        
        days_to_censor = (outcome_date - event['admit_date']).days
         
        num_time_steps, max_time = len(month_endpoints), max(month_endpoints)
        y = days_to_censor < (max_time * 30) and outcome
        y_seq = np.zeros(num_time_steps)
        time_at_event = min([i for i, mo in enumerate(month_endpoints)
                               if days_to_censor < (mo * 30)]) if days_to_censor < (max_time * 30) else num_time_steps - 1
        if y:
             y_seq[time_at_event:] = 1
        y_mask = np.array([1] * (time_at_event + 1) + [0] * (num_time_steps - (time_at_event + 1)))
        
        assert time_at_event >= 0 and len(y_seq) == len(y_mask)
        return y, y_seq.astype('float64'), y_mask.astype('float64'), time_at_event, days_to_censor


In [144]:
get_label1(events[761], index_date, [0,2])

(True, array([0., 1.]), array([1., 1.]), 1, 7)

In [149]:
get_label1(events[761], index_date, [0])

(False, array([0.]), array([1.]), 0, 7)

In [145]:
get_label1(events[761], index_date, [2])

(True, array([1.]), array([1.]), 0, 7)

In [60]:
events[224]

{'admdate': '2009-08-04',
 'admid': '0022714578',
 'codes': '197',
 'admit_date': datetime.datetime(2009, 8, 4, 0, 0)}

In [ ]:
            y, y_seq, y_mask, time_at_event, days_to_censor = self.get_label1(patient, idx)


In [133]:
if outcome == True and lookback > outcome_date:
    lookback = outcome_date 
            

In [134]:
lookback

datetime.datetime(2015, 4, 10, 0, 0)

In [ ]:
enumerate(events)

In [ ]:
for i in events:
    date = events['admdate]
                     

In [ ]:
avai_indices, gold = get_avai_trajectory_indices1 (patient_dict, events, feat_subgroup, split_group)


In [ ]:
patient_dict.update({'avai_indices': avai_indices, 'y': gold})
patient_dict.update({'events': events})

In [ ]:
patient_id

In [ ]:
O = []

In [ ]:
obs_time_end

In [ ]:
index_date

In [ ]:
outcome_date

In [ ]:
patient = patient_dict

In [ ]:
len( patient['events'])

In [ ]:
num_time_steps, max_time = len([1,3,6,12]), max([1,3,6,12])

In [ ]:
y = days_to_censor < (max_time * 30) and patient['outcome']


In [ ]:
        y_seq = np.zeros(num_time_steps)


In [ ]:
days_to_censor

In [ ]:
index_date

In [ ]:
days_to_censor

In [ ]:
 (max_time * 30)

In [ ]:
days_to_censor = (patient['outcome_date'] - index_date).days
num_time_steps, max_time = len([1,3,6,12]), max([1,3,6,12])
y = patient['outcome']
y_seq = np.zeros(num_time_steps)
time_at_event = min([i for i, mo in enumerate([1,3,6,12])
                     if days_to_censor < (mo * 30)]) if days_to_censor < (max_time * 30) else num_time_steps 

In [ ]:
time_at_event

In [ ]:
if y:
    y_seq[time_at_event:] = 1
y_mask = np.array([1] * (time_at_event ) + [0] * (num_time_steps - (time_at_event )))


In [ ]:
y

In [ ]:
y_mask

In [ ]:
y_seq

In [ ]:
num_time_steps, max_time = len([1,3,6,12]), max([1,3,6,12])

In [ ]:
y_mask

In [ ]:
 np.array([1] * (time_at_event + 1) + [0] * (num_time_steps - (time_at_event + 1)))

In [ ]:
max_time

In [ ]:
days_to_censor

In [ ]:
max_time*30

In [ ]:
time_at_event

In [ ]:
    def time_label(self, patient, mo1,mo3,mo6):
        '''
            outcome_date: outcome occurance or END_OF_TIME_DATE
            time_at_event: years to outcome_data/END_OF_TIME_DATE
            y_seq: zero array unless ever_develops_outcome => y_seq[time_at_event:]=1
                    * Used as golds in cumulative_probability_layer
            y_mask: how many years left in the disease window,
                    ([1] for 0:time_at_event years and [0] for the rest)
                    (without linear interpolation, y_mask looks like complement of y_seq)
                    * Used for mask loss in cumulative_probability_layer
            Ex1:  a patient has the outcome in 3 years
                    time_at_event: 2,
                    y_seq: [0, 0, 1, 1, 1]
                    y_mask: [1, 1, 1, 0, 0]
            Ex2:  a patient never gets outcome
                    y_seq: [0, 0, 0, 0, 0]
                    y_mask: [1, 1, 0, 0, 0]
                    time_at_event: 1,
        '''
        event1 = patient['events'][mo1]
        event3 = patient['events'][mo3]
        event6 = patient['events'][mo6]
        
        
        days_to_censor1 = (patient['outcome_date'] - event1 ['admit_date']).days
        days_to_censor2 = (patient['outcome_date'] - event3 ['admit_date']).days
        days_to_censor3 = (patient['outcome_date'] - event6 ['admit_date']).days
         
        num_time_steps, max_time = len(self.args.month_endpoints), max(self.args.month_endpoints)
        y = days_to_censor < (max_time * 30) and patient['outcome']
        y_seq = np.zeros(num_time_steps)
        time_at_event = min([i for i, mo in enumerate(self.args.month_endpoints)
                             if days_to_censor < (mo * 30)]) if days_to_censor < (max_time * 30) else num_time_steps - 1
        if y:
            y_seq[time_at_event:] = 1
        y_mask = np.array([1] * (time_at_event + 1) + [0] * (num_time_steps - (time_at_event + 1)))
        
        assert time_at_event >= 0 and len(y_seq) == len(y_mask)
        return y, y_seq.astype('float64'), y_mask.astype('float64'), time_at_event, days_to_censor

In [ ]:
patient_id

In [ ]:
patient_metadata.keys()

In [ ]:
patient_id

In [ ]:
patient_dict = {'patient_id': patient_id}

In [ ]:
patient_dict.update({'events': events})


In [ ]:
feat_subgroup = {'birth_date': patient_metadata[patient_id]['birthdate']}


In [ ]:
feat_subgroup

In [ ]:
 patient_dict.update({'outcome': outcome,
                                 'dob': dob,
                                 'outcome_date': outcome_date,
                                 'split_group': patient_metadata[patient_id]['split_group'],
                                 'obs_time_end': obs_time_end,
                                 'index_date': index_date,
                                 'dx_date': dx_date,
                                 'ks': ks,
                                 'sex': sex,
                                 'bmi': bmi,
                                 'race': race})

In [ ]:
patient_metadata[patient_id]['split_group']

In [ ]:
split_group = 'dev'

In [ ]:
avai_indices, gold = get_avai_trajectory_indices(patient_dict, events, feat_subgroup, split_group, args)


In [ ]:
patient_dict.update({'avai_indices': avai_indices, 'y': gold})


In [ ]:
patients = np.array([])

In [ ]:
patients = np.append(patients, patient_dict)

In [ ]:
patient = patients[0]

In [ ]:
selected_idx = [random.choice(patient['avai_indices']) for _ in range (4)]


In [ ]:
patient['avai_indices']

In [ ]:
selected_idx

In [ ]:
idx = selected_idx[0]

In [ ]:
events_to_date = patient['events'][: selected_idx[2] + 1]

In [ ]:
len(events_to_date)

In [ ]:
ev_samp =  patient['events'] [0:20]

In [ ]:
ev_samp [: 3 +1]

In [ ]:
ev_samp[: 17 + 1]

In [ ]:
ev_samp[: 13 + 1]

In [ ]:
patient['events'][2150:2200]

In [ ]:
events_to_date[1:10]

In [ ]:
outcome

In [ ]:
def get_time_seq1( events, reference_date):
    deltas = np.array([abs((reference_date - event['admit_date']).days) for event in events])
    multipliers = 2 * np.pi / (np.linspace(start=MIN_TIME_EMBED_PERIOD_IN_DAYS, stop=MAX_TIME_EMBED_PERIOD_IN_DAYS,
                                           num=self.args.time_embed_dim))

    deltas, multipliers = deltas.reshape(len(deltas), 1), multipliers.reshape(1, len(multipliers))
    positional_embeddings = np.cos(deltas * multipliers)
    return max(deltas), positional_embeddings